In [1]:
import svfpy
import rasterio
import numpy as np
import zarr

In [2]:
mds = '../test/data/mds_sao_paulo_city_1000.tiff'

In [3]:
svf = svfpy.SVF(mds_path=mds, kernel_size=15)

In [4]:
profile = svf.mds_src.profile

In [5]:
svf_result = svfpy.calculate(svf)
# points, points_sem_obs = svfpy.calculate(svf)

(0, 8)
Calculando arctan
(8, 12)
Calculando arctan
(12, 16)
Calculando arctan
(16, 20)
Calculando arctan
(20, 24)
Calculando arctan
(24, 28)
Calculando arctan
(28, 32)
Calculando arctan
(32, 36)
Calculando arctan
(36, 40)
Calculando arctan
Calulando max ...
Calculando SVF ..


In [6]:
# np.sum(svf_result, axis=2).shape
# svfpy._fuse_lines(svf)
# svfpy.distance_matrix(size=17).shape

In [7]:
with rasterio.open(f'../tmp/result.tif', 'w', **profile) as dst:
    dst.write(svf_result, 1)

In [8]:
# for i in np.arange(40):
#     with rasterio.open(f'../tmp/result{i}.tif', 'w', **profile) as dst:
#         dst.write(result_temp[:,:,i,1], 1)

In [65]:
Window.from_slices((0,30000), (0, 1024))

Window(col_off=0, row_off=0, width=1024, height=30000)

In [15]:
for k, m in enumerate(mdss):
    # transform = transformations[k]
    m = np.mean(m[0:1000, 0:1000, :, :].reshape(1000,1000, 121), axis=2)
    profile.update(
        # transform=transform,
        # width=m.shape[0],
        # height=m.shape[1]
    )

    with rasterio.open(f'../tmp/11-11-{k}-{m.shape[1]}.tif', 'w', **profile) as dst:
        dst.write(m, 1)